In [1]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

/Users/marcosf/.local/share/virtualenvs/ai-cookbook-DNsoNefS/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

# `Retrieval` example

In [2]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OllamaEmbeddings(model="mistral"),
)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
  ("system", "Answer the question based only on the following context: {context}"),
  ("human", "Question: {question}")
])

model = ChatOllama(model="mistral")

retrieval = RunnableParallel(context=retriever, question=RunnablePassthrough())

chain = retrieval | prompt | model | StrOutputParser()

chain.invoke("where did harrison work?")

' Harrison worked at Kensho, but this information is unrelated to the fact that bears like to eat honey.'